In [1]:
!pip install deep-sort-realtime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 67.2 MB/s eta 0:00:00


In [2]:
import torch
import torchvision
import cv2
import numpy as np
from deep_sort_realtime.deepsort_tracker import DeepSort
import warnings
warnings.filterwarnings('ignore')

In [3]:
!pip install -qr https://raw.githubusercontent.com/ultralytics/yolov5/master/requirements.txt  # install dependencies

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dask-cudf 21.12.2 requires cupy-cuda115, which is not installed.
cudf 21.12.2 requires cupy-cuda115, which is not installed.
tensorflow 2.11.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.3 which is incompatible.
tensorflow-transform 1.12.0 requires pyarrow<7,>=6, but you have pyarrow 5.0.0 which is incompatible.
tensorflow-serving-api 2.11.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.3 which is incompatible.
librosa 0.10.0 requires soundfile>=0.12.1, but you have soundfile 0.11.0 which is incompatible.
distributed 2021.11.2 requires dask==2021.11.2, but you have dask 2022.2.0 which is incompatible.
dask-cudf 21.12.2 requires dask<=2021.11.2,>=2021.11.1, but you have dask 2022.2.0 which is incompatible.
cloud-tpu-client 0.10 requires google-api-python-client==1.8.0, but you have googl

In [4]:
# Load YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
requirements: YOLOv5 requirement "setuptools>=65.5.1" not found, attempting AutoUpdate...

requirements: 1 package updated per /root/.cache/torch/hub/ultralytics_yolov5_master/requirements.txt
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

YOLOv5 🚀 2023-3-22 Python-3.7.12 torch-1.13.0 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)



  0%|          | 0.00/14.1M [00:00<?, ?B/s]


Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [5]:
object_tracker = DeepSort()

In [6]:
class Car_Tracker:
    
    def __init__(self):
        pass
    
    def detect_cars(self, image, model, threshold=0.5):
        # Use a pre-trained model to predict object bounding boxes and class probabilities in the input image
        predictions = model(image)
        
        # Select only the predicted bounding boxes and class probabilities for cars
        car_predictions = predictions.pred[0][predictions.pred[0][:, 5] == 2]
        
        # Filter out low-confidence detections
        detections = [(box.cpu().numpy(), confidence.cpu().numpy()) for box, confidence in zip(car_predictions[:, :4], car_predictions[:, 4]) if confidence > threshold]
        
        # Draw bounding boxes around detected cars
        for box, _ in detections:
            cv2.rectangle(image, (int(box[0]), int(box[1])), (int(box[2]), int(box[3])), (0, 255, 0), 3)
            cv2.putText(image, "car", (int(box[0])-10, int(box[1])-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        return detections
    
    def update_tracks(self, tracks, count, track_ids):
        # Loop over all tracks
        for track in tracks:
            # If track is not yet confirmed, skip to the next one
            if not track.is_confirmed():
                continue

            # Get the ID of the track
            track_id = track.track_id

            # If the track ID is not already in the list of track IDs, 
            # increment the count and add the track ID to the list
            if track_id not in track_ids:
                count += 1
                track_ids.append(track_id)

            # Convert the track's bounding box to a ltrb format
            bbox = track.to_ltrb()

        # Return the updated list of track IDs and the count
        return track_ids, count


In [7]:
CT = Car_Tracker()

In [8]:
# Opening the video file for reading
vidcap = cv2.VideoCapture('/kaggle/input/car-video/video1.mp4')

# Initializing variables for processing frames
success = True
count2 = 0
img_array = []
track_ids = []
count = 0 

In [9]:
# Looping through each frame of the video file
while success:
    # Reading the next frame from the video file
    success,image = vidcap.read()
    
    # Checking if the frame was successfully read
    if(success == 0 or image is None): 
        break

    # Detect cars in the current frame using a pre-trained model
    detections = CT.detect_cars(image, model, 0.5)
    
    # Update the tracks of the detected cars using an object tracker
    tracks = object_tracker.update_tracks(detections, frame=image)
    
    # Update the count of cars detected and their track IDs
    track_ids, count = CT.update_tracks(tracks, count, track_ids)
    
    # Add the count of detected cars as text to the current frame
    cv2.putText(image, f'Number of Cars: {int(count)}', (20, 100), cv2.FONT_HERSHEY_SIMPLEX, 3, (255,255,255), 3)
    
    # Add the current frame to an array of frames
    img_array.append(image)
    
    # If the current frame is one of the specified frames, save it as a JPEG file
    if(count2 == 12 or count2 == 490 or count2 == 200 or count2 == 400):
        cv2.imwrite("frame%d.jpg" % count2, img_array[-1])     # save frame as JPEG file
        
    # Increment the frame counter
    count2 += 1

In [10]:
# Determining the size of the frames and initializing a VideoWriter object for writing the output video file
height, width, layers = img_array[0].shape
size = (width,height)
out = cv2.VideoWriter('project_YOLO.avi',cv2.VideoWriter_fourcc(*'DIVX'), 15, size)
 
# Writing each frame of the output video file
for i in range(len(img_array)):
    out.write(img_array[i])
    
# Releasing the VideoWriter object
out.release()